Notebook to determine packing fraction for lattice structure

Range of pores from 2D: 

0.017 - 0.024

In [2]:
import numpy as np
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt

'''
For a given number of voids in the x direction and the radius of the voids, compute the 
positions of the voids and the interstitial space between them. The space should be greater than 
the mesh maximum size.
'''


def write_gmsh(filename, positions, radius, domain, h):
    # # --- Gmsh .geo script output ---
    with open(f'{filename}', 'w') as f:
        f.write('SetFactory("OpenCASCADE");\n')
        f.write('Mesh.MshFileVersion = 2.0;\n\n')

        # Box
        f.write(f'Box(1) = {{0, 0, 0, {domain[0]}, {domain[1]}, {domain[2]}}};\n\n')

        void_tags = []
        for i, (x, y, z) in enumerate(positions):
            tag = i + 2  # start after box ID 1
            void_tags.append(tag)
            f.write(f'Sphere({tag}) = {{{x}, {y}, {z}, {radius}}};\n')

        void_list = ', '.join(str(tag) for tag in void_tags)

        f.write('\n// --- Subtract voids ---\n')
        f.write(f'BooleanDifference{{ Volume{{1}}; Delete; }}{{ Volume{{{void_list}}}; Delete; }}\n\n')

        f.write('// --- Tag fluid volume ---\n')
        f.write('volumes() = Volume{:};\n')
        f.write('Physical Volume(\"Fluid\") = {volumes};\n\n')

        # No box surface tagging here (as requested)

        # Tag void surfaces together
        f.write('// --- Void surfaces ---\n')
        f.write('voidSurfaces[] = {};\n')
        for (x, y, z) in positions:
            rpad = radius + 1e-3
            f.write(
            f'voidSurfaces[] += Surface In BoundingBox{{{x - rpad}, {y - rpad}, {z - rpad}, {x + rpad}, {y + rpad}, {z + rpad}}};\n'
            )
        f.write('Physical Surface(1000) = {voidSurfaces[]};\n\n')

        f.write('Mesh.Algorithm = 6;\n')
        f.write(f'Mesh.MeshSizeMax = {h};\n')


import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

def plot_voids(positions, radius, domain):
    def create_sphere(center, radius, resolution=20):
        u, v = np.meshgrid(np.linspace(0, 2 * np.pi, resolution),
                           np.linspace(0, np.pi, resolution))
        x = center[0] + radius * np.cos(u) * np.sin(v)
        y = center[1] + radius * np.sin(u) * np.sin(v)
        z = center[2] + radius * np.cos(v)
        return x, y, z

    def draw_domain_box(ax, domain):
        Lx, Ly, Lz = domain
        # Define 8 corners of the box
        corners = np.array([
            [0, 0, 0], [Lx, 0, 0], [Lx, Ly, 0], [0, Ly, 0],
            [0, 0, Lz], [Lx, 0, Lz], [Lx, Ly, Lz], [0, Ly, Lz]
        ])
        # Define lines between corners (12 edges)
        edges = [
            (0, 1), (1, 2), (2, 3), (3, 0),  # bottom
            (4, 5), (5, 6), (6, 7), (7, 4),  # top
            (0, 4), (1, 5), (2, 6), (3, 7)   # vertical
        ]
        for start, end in edges:
            ax.plot(*zip(corners[start], corners[end]), color='black', linewidth=1)

    # Plotting
    fig = plt.figure(figsize=(8, 6))
    ax = fig.add_subplot(111, projection='3d')

    for pos in positions:
        x, y, z = create_sphere(pos, radius)
        ax.plot_surface(x, y, z, color='steelblue', edgecolor='k', linewidth=0.2, alpha=0.7)

    draw_domain_box(ax, domain)

    ax.set_xlim(0, domain[0])
    ax.set_ylim(0, domain[1])
    ax.set_zlim(0, domain[2])
    ax.set_box_aspect(domain)
    # ax.set_title("BCC Lattice with Spherical Voids and Domain Wireframe")
    
    # Make axes numbers bold and set size
    for axis in [ax.xaxis, ax.yaxis, ax.zaxis]:
        axis.set_tick_params(labelsize=10)
        for tick in axis.get_major_ticks():
            tick.label1.set_fontweight('bold')
    
    plt.tight_layout()
    # plt.show()
    plt.savefig('lattice_rp_06662_pf_2341.pdf',  format='pdf', bbox_inches='tight')
    # plt.savefig('lattice_rp_8199_pf_4363.svg')

import numpy as np

# def compute_bcc_positions(r, h, domain):
#     """
#     Compute BCC positions ensuring:
#     - The gap between spheres = i = a - 2r
#     - The domain edges are at distance i from nearest sphere surface
#     - i > 3h

#     Returns:
#         positions: array of (x, y, z)
#         a: spacing
#         i: gap between spheres and from spheres to walls
#         (nx, ny, nz): number of unit cells
#     """
#     Lx, Ly, Lz = domain
#     min_i = 3 * h

#     # Solve for max nx such that i = a - 2r > min_i
#     def get_max_n(L):
#         for n in range(1, 100):
#             a = L / (n + 1)
#             i = a - 2 * r
#             if i <= min_i:
#                 return n - 1  # previous was max valid
#         raise ValueError("Could not find suitable number of voids")

#     nx = get_max_n(Lx)
#     ny = get_max_n(Ly)
#     nz = get_max_n(Lz)

#     ax = Lx / (nx + 1)
#     ay = Ly / (ny + 1)
#     az = Lz / (nz + 1)

#     ix = ax - 2 * r
#     iy = ay - 2 * r
#     iz = az - 2 * r

#     # Use average a and i for isotropic spheres (or keep them separate if you want anisotropic spacing)
#     assert abs(ix - iy) < 1e-8 and abs(iy - iz) < 1e-8, "Anisotropic spacing detected"
#     a = ax
#     i = ix

#     centers = []
#     for ix_ in range(nx):
#         for iy_ in range(ny):
#             for iz_ in range(nz):
#                 x0 = r + i + ix_ * a
#                 y0 = r + i + iy_ * a
#                 z0 = r + i + iz_ * a
#                 centers.append((x0, y0, z0))  # corner

#                 # Body center
#                 x1 = x0 + a / 2
#                 y1 = y0 + a / 2
#                 z1 = z0 + a / 2
#                 if x1 + r + i <= Lx and y1 + r + i <= Ly and z1 + r + i <= Lz:
#                     centers.append((x1, y1, z1))

#     return np.array(centers), a, i, (nx, ny, nz)

import numpy as np

def compute_bcc_positions(r, h, domain):
    def find_max_n(L):
        for n in range(1, 1000):
            i = (L - 2 * r * n) / (n + 1)
            if i <= 3 * h:
                return n - 1
        raise ValueError("Domain too small or h too large")

    Lx, Ly, Lz = domain
    nx = find_max_n(Lx)
    ny = find_max_n(Ly)
    nz = find_max_n(Lz)

    ix = (Lx - 2 * r * nx) / (nx + 1)
    iy = (Ly - 2 * r * ny) / (ny + 1)
    iz = (Lz - 2 * r * nz) / (nz + 1)

    ax = 2 * r + ix
    ay = 2 * r + iy
    az = 2 * r + iz

    x_base = [ix + r + i * ax for i in range(nx)]
    y_base = [iy + r + j * ay for j in range(ny)]
    z_base = [iz + r + k * az for k in range(nz)]

    positions = []

    # Corner spheres
    for x in x_base:
        for y in y_base:
            for z in z_base:
                positions.append((x, y, z))

                # Body-centered sphere
                xc = x + ax / 2
                yc = y + ay / 2
                zc = z + az / 2
                if (xc + r + ix <= Lx) and (yc + r + iy <= Ly) and (zc + r + iz <= Lz):
                    positions.append((xc, yc, zc))

    return np.array(positions), (nx, ny, nz), (ax, ay, az), (ix, iy, iz)


# Example usage
r = 0.058
h = 0.01
domain = (1.0, 1.0, 1.0)

positions, a, i, (nx, ny, nz) = compute_bcc_positions(r, h, domain)
# plot_voids(positions, r, domain)

num_voids = len(positions)
print(f'Num voids: {num_voids}')
volume_voids = num_voids * (4/3) * np.pi * r**3
packing_fraction = volume_voids / (domain[0] * domain[1] * domain[2])
packing_percentage = int(packing_fraction * 100)

print(f'Packing fraction: {packing_fraction:.4f}')

# plot_voids(positions, r, domain)

# write_gmsh(f'bcc-lattice-pf_{packing_percentage}.geo', positions, r, domain, h)


Num voids: 341
Packing fraction: 0.2787


Packing fraction from 2D: 0.20 - 0.46

h = 0.005
For packing fraction of 0.2: r = 0.012

For packing fraction of 0.44: r = 0.03

In [ ]:
import numpy as np

def compute_bcc_positions(r, h, domain):
    def find_max_n(L):
        for n in range(1, 1000):
            gap = (L - 2.0*r*n) / (n + 1)
            if gap <= 3.0*h:
                return n - 1
        raise ValueError("r too large or h too big")

    Lx, Ly, Lz = domain
    nx = find_max_n(Lx)
    ny = find_max_n(Ly)
    nz = find_max_n(Lz)

    ix = (Lx - 2.0*r*nx) / (nx + 1)
    iy = (Ly - 2.0*r*ny) / (ny + 1)
    iz = (Lz - 2.0*r*nz) / (nz + 1)

    ax, ay, az = 2*r + ix, 2*r + iy, 2*r + iz

    positions = []
    x0s = [ix + r + i*ax for i in range(nx)]
    y0s = [iy + r + j*ay for j in range(ny)]
    z0s = [iz + r + k*az for k in range(nz)]

    for x0 in x0s:
        for y0 in y0s:
            for z0 in z0s:
                positions.append((x0, y0, z0))
                # body center
                xc, yc, zc = x0 + ax/2, y0 + ay/2, z0 + az/2
                if (xc + r + ix <= Lx) and (yc + r + iy <= Ly) and (zc + r + iz <= Lz):
                    positions.append((xc, yc, zc))

    return np.array(positions)


# ── PARAMETERS ────────────────────────────────────────────────────────────────
domain       = (1.0, 1.0, 1.0)
h            = 0.01
r_min, r_max = 0.01, 0.06
max_voids    = 500
pf_min, pf_max = 0.21, 0.46
N_candidates = 50000      # total r’s to try
report_every = 1000       # print status every this many tries
target_n     = 300        # how many final solutions to keep
# ──────────────────────────────────────────────────────────────────────────────

domain_vol = np.prod(domain)
# linear grid; you could also do np.random.uniform(r_min, r_max, N_candidates)
r_vals = np.linspace(r_min, r_max, N_candidates)

results = []
attempts = 0
pfs = []

for r in r_vals:
    attempts += 1
    try:
        pos = compute_bcc_positions(r, h, domain)
    except ValueError:
        continue

    nv = len(pos)
    if nv > max_voids:
        continue

    pf = nv * (4.0/3.0)*np.pi * r**3 / domain_vol
    if pf_min <= pf <= pf_max:
        results.append((r, nv, pf))

    if attempts % report_every == 0:
        print(f"  Tried {attempts} r’s, found {len(results)} candidates so far.")
        # print(f'Pfs: {pf}')
        pfs.append(pf)

# ── SELECT 100 SPANNING THE FULL PF RANGE ─────────────────────────────────────
if len(results) < target_n:
    print(f"Only {len(results)} candidates found; try upping N_candidates or broadening the pf window.")
else:
    # sort by pf
    results.sort(key=lambda x: x[2])
    # pick evenly‐spaced indices
    idxs = np.linspace(0, len(results)-1, target_n, dtype=int)
    selected = [results[i] for i in idxs]

    print(f"\nFinal {target_n} solutions spanning PF {selected[0][2]:.4f} → {selected[-1][2]:.4f}:\n")
    for i, (r, nv, pf) in enumerate(selected, 1):
        print(f"{i:3d}: r = {r:.5f}, voids = {nv:3d}, packing = {pf:.4f}")
        
print(f'Max PF: {max(pf)}')
print(f'Min PF: {min(pf)}')

In [2]:
import numpy as np
import os
import csv

def compute_bcc_positions(r, h, domain):
    def find_max_n(L):
        for n in range(1, 1000):
            gap = (L - 2.0*r*n) / (n + 1)
            if gap <= 3.0*h:
                return n - 1
        raise ValueError("r too large or h too big")

    Lx, Ly, Lz = domain
    nx = find_max_n(Lx)
    ny = find_max_n(Ly)
    nz = find_max_n(Lz)

    ix = (Lx - 2.0*r*nx) / (nx + 1)
    iy = (Ly - 2.0*r*ny) / (ny + 1)
    iz = (Lz - 2.0*r*nz) / (nz + 1)

    ax, ay, az = 2*r + ix, 2*r + iy, 2*r + iz

    positions = []
    x0s = [ix + r + i*ax for i in range(nx)]
    y0s = [iy + r + j*ay for j in range(ny)]
    z0s = [iz + r + k*az for k in range(nz)]

    for x0 in x0s:
        for y0 in y0s:
            for z0 in z0s:
                positions.append((x0, y0, z0))
                # body center
                xc, yc, zc = x0 + ax/2, y0 + ay/2, z0 + az/2
                if (xc + r + ix <= Lx) and (yc + r + iy <= Ly) and (zc + r + iz <= Lz):
                    positions.append((xc, yc, zc))

    return np.array(positions)


# ── PARAMETERS ────────────────────────────────────────────────────────────────
domain         = (1.0, 1.0, 1.0)
h              = 0.01
r_min, r_max   = 0.01, 0.06
max_voids      = 400
pf_min, pf_max = 0.21, 0.46
N_candidates   = 50000      # total r’s to try
report_every   = 1000       # print status every this many tries
target_n       = 100        # how many final solutions to keep

# for the CSV output:
base_path = "/mnt/ssd1/joshgregory/clotsimnet_poop"   # ← change this!
out_csv   = "tasks_poop.csv"
# ──────────────────────────────────────────────────────────────────────────────

domain_vol = np.prod(domain)
r_vals     = np.linspace(r_min, r_max, N_candidates)

# 1) Find all candidates
results = []
pfs = []
attempts = 0
for r in r_vals:
    attempts += 1
    try:
        pos = compute_bcc_positions(r, h, domain)
    except ValueError:
        continue

    nv = len(pos)
    if nv > max_voids:
        continue

    pf = nv * (4.0/3.0)*np.pi * r**3 / domain_vol
    if pf_min <= pf <= pf_max:
        results.append((r, nv, pf))
        pfs.append(pf)

    if attempts % report_every == 0:
        print(f"  Tried {attempts} r's, found {len(results)} candidates so far.")

# 2) Select 500 spanning the PF range
if len(results) < target_n:
    raise RuntimeError(f"Only {len(results)} candidates found; "
                       "increase N_candidates or broaden PF window.")
results.sort(key=lambda x: x[2])
idxs = np.linspace(0, len(results)-1, target_n, dtype=int)
selected = [results[i] for i in idxs]  # Keep full tuples (r, nv, pf)

print(f"\nCollected {len(selected)} configurations spanning PF "
      f"{selected[0][2]:.4f} → {selected[-1][2]:.4f}.")

# 3) Write to CSV
with open(out_csv, "w", newline="") as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(["index", "case_dir", "sim_id", "r_p"])
    for idx, (r, _, _) in enumerate(selected):  # Only need r for the CSV
        sim_str = f"{int(r * 1e5):05d}"
        case_dir = os.path.join(base_path, f"bcc_lattice_rp_{sim_str}")
        sim_id = f"bcc_lattice_rp_{sim_str}"
        writer.writerow([idx, case_dir, sim_id, r])

# Print summary
for i, (r, nv, pf) in enumerate(selected, 1):
    print(f"{i:3d}: r = {r:.5f}, voids = {nv:3d}, packing = {pf:.4f}")

print(f"Wrote {len(selected)} tasks to '{out_csv}'")
print(f'Max PF: {max(pf)}')
print(f'Min PF: {min(pf)}')

  Tried 45000 r's, found 715 candidates so far.
  Tried 46000 r's, found 1715 candidates so far.
  Tried 47000 r's, found 2715 candidates so far.
  Tried 48000 r's, found 3715 candidates so far.
  Tried 49000 r's, found 4715 candidates so far.
  Tried 50000 r's, found 5715 candidates so far.

Collected 100 configurations spanning PF 0.2285 → 0.3085.
  1: r = 0.05429, voids = 341, packing = 0.2285
  2: r = 0.05434, voids = 341, packing = 0.2292
  3: r = 0.05440, voids = 341, packing = 0.2300
  4: r = 0.05446, voids = 341, packing = 0.2307
  5: r = 0.05452, voids = 341, packing = 0.2314
  6: r = 0.05457, voids = 341, packing = 0.2322
  7: r = 0.05463, voids = 341, packing = 0.2329
  8: r = 0.05469, voids = 341, packing = 0.2336
  9: r = 0.05475, voids = 341, packing = 0.2344
 10: r = 0.05480, voids = 341, packing = 0.2351
 11: r = 0.05486, voids = 341, packing = 0.2359
 12: r = 0.05492, voids = 341, packing = 0.2366
 13: r = 0.05498, voids = 341, packing = 0.2374
 14: r = 0.05504, voids 

TypeError: 'numpy.float64' object is not iterable